In [240]:

import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
#from gridworld import GridworldEnv


In [277]:
# Minataur blocked paths

Blocked_minataur = {
    0: [x for x in range(0,6)],
    1: [5,11,17,23,29],
    2: [x for x in range(24,30)],
    3: [0,6,12,18,24]
}

Blocked_agent = {
    0: [16,17,25,26,27,28,0,1,2,3,4,5],
    1: [1,7,13,9,15,27,5,11,17,23,29],
    2: [10,11,19,20,21,22,24,25,26,27,28,29],
    3: [2,8,14,10,16,28,0,6,12,18,24]
}

agent_moves = {}
minataur_moves = {}

grid = np.arange(30).reshape((5,6))
it = np.nditer(grid, flags=['multi_index'])

while not it.finished:
    s = it.iterindex
    x,y  = it.multi_index
    
    moves_agent = []
    moves_minataur = []
    for move in range(4):
        
        if not s in Blocked_agent[move]:
            moves_agent.append(move)
        
        if not s in Blocked_minataur[move]:
            moves_minataur.append(move)
            
    moves_agent.append(4)
    
    agent_moves[s] = moves_agent
    minataur_moves[s] = moves_minataur
    
    it.iternext()



In [302]:
import numpy as np
import sys
from gym.envs.toy_text import discrete

STAY = 0
UP = 1
RIGHT = 2
DOWN = 3
LEFT = 4


class GridworldEnv(discrete.DiscreteEnv):
    """
    You are an agent on an MxN grid and your goal is to reach the terminal
    state at the bottom right corner. There is also an minatours that is moving randomly
    unifromaly, if he catches you, you will be eated. 

    You can take actions in each direction (UP=0, RIGHT=1, DOWN=2, LEFT=3, STAY=4).
    Actions going off the edge leave you in your current state.
    Actions going into a wall, leave you in your current state. 
    You receive a reward of -1 at each step until you reach a terminal state.
    Your recive a reward of -100 if the minatours catches you
    
    """

    metadata = {'render.modes': ['human', 'ansi']}

    def __init__(self, shape=[5,6]):

        self.shape = shape

        nS = 900
        nA = 5

        MAX_Y = shape[0]
        MAX_X = shape[1]
    
        #Translation Matrix
        state_index = np.arange(nS).reshape((30,30))
        
        # Minataur blocked paths
        Edge_up = [x for x in range(0,6)]
        Edge_right = [5,11,17,23,29]
        Edge_down = [x for x in range(24,30)]
        Edge_left = [0,6,12,18,24]
        

        #Agents blocked paths
        Blocked_agent_up = [16,17,25,26,27,28,0,1,2,3,4,5]
        Blocked_agent_right = [1,7,13,9,15,27,5,11,17,23,29]
        Blocked_agent_down = [10,11,19,20,21,22,24,25,26,27,28,29]
        Blocked_agent_left = [2,8,14,10,16,28,0,6,12,18,24]
        
        P = {}

        #Matrix (30*30) with all numbered possible states
        grid = np.arange(nS).reshape((30,30))
        #Iteration tool, instead of using two for loops
        it = np.nditer(grid, flags=['multi_index'])

        while not it.finished:

            s = it.iterindex
          
            x,y  = it.multi_index

            #Stopping condition, when have reached terminal state (either end of maze or eaten)
            is_done = lambda x: x == 28
            is_eaten = lambda x,y: x == y
            #Negative reward for everytime step we are not in the terminal state
            reward = 0.0 if is_done(x) else -1.0
            #Negative reward if out state and the minataurs is the same(we have been eaten)
            if is_eaten(x,y) == True:
                reward = -1

            #print("Index of state:", index)
            
                        
            P[s] = {a : [[]] for a in range(nA)}

            # We're stuck in a terminal state if the agent is in the terminal node, no matter where
            # the minataur is
            if is_done(x) or is_eaten(x,y):
                P[s][UP] = [([1.0], [s], [reward], True)]
                P[s][RIGHT] = [([1.0], [s], [reward], True)]
                P[s][DOWN] = [([1.0], [s], [reward], True)]
                P[s][LEFT] = [([1.0], [s], [reward], True)]
                P[s][STAY] = [([1.0], [s], [reward], True)]
            # Not a terminal state
            else:
                # First determine states depending on out choice of action
                #If we go up when we are at the top of maze, we end up the same pos.
                #If we go up when we are at any position in Walls_Up, we end up in same pos
                ns_up = x if x in Blocked_agent_up else x - MAX_X
                ns_right = x if x in Blocked_agent_right else x + 1
                ns_down = x if x in Blocked_agent_down else x + MAX_X
                ns_left = x if x in Blocked_agent_left else x - 1

                # Determine next state of the minataurs, 4 possibilies
                ns_m_up = y if y in Edge_up else y - MAX_X
                ns_m_right = y if y in Edge_right else y + 1
                ns_m_down = y if y in Edge_down else y + MAX_X
                ns_m_left = y if y in Edge_left else y -1

                # After an action a, we can be in four possible states. A total of 16 
                
                list1 = [x,ns_up, ns_right, ns_down, ns_left]
                list2 = [y,ns_m_up, ns_m_right,ns_m_down, ns_m_left]
     
                action = 0
        
                for ns in list1:
                    prob_list = []
                    next_state_list = []
                    reward_list = []

                    for ns_m in list2:
                        reward = -1
                        ns_index = state_index[ns][ns_m]
                        next_state_list.append(ns_index)
                        prob_list.append(0.25)
                        
                        if ns == ns_m:
                            reward=-50
                        if ns ==y:
                            reward = -50
                        
                        #if action == 4:
                        #    reward = -0.9

                        reward_list.append(reward)
                    
                    
                    P[s][action] = [(prob_list,next_state_list, reward_list, is_done(ns))] 
                    action += 1
                    

            it.iternext()

        # Initial state distribution is uniform
        isd = np.ones(nS) / nS

        # We expose the model of the environment for educational purposes
        # This should not be used in any model-free learning algorithm
        self.P = P

        super(GridworldEnv, self).__init__(nS, nA, P, isd)


In [303]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [304]:

policy, v = value_iteration(env)

#print("Policy Probability Distribution:")
#print(policy)
#print("")

#print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
#print(np.reshape(np.argmax(policy, axis=1), (30,30)))
#print("")

#print("Value Function:")
#print(v)
#print("")

#print("Reshaped Grid Value Function:")
#print(v.reshape((30,30)))
#print("")


pol = np.reshape(np.argmax(policy, axis=1), (30,30))
pol[:,13].reshape((5,6))


14.299199999999999
7.717475840000002
4.911613618176002
3.8355623727104025
3.3701874608668927
2.942129985484179
2.745033525083741
2.63674204302173
2.5071180084190274
2.3419993491561435
2.003428861003542
1.7251384061422215
1.4966763996774972
1.290543651172399
1.1070173746931893
0.9382924071209331
0.7925864375071257
0.6679748990804697
0.5620065013847366
0.47222752196004336
0.3963671980727881
0.3323969050446749
0.2785386391014981
0.23325226159279566
0.1952141397933076
0.16329305063859323
0.13652619441325697
0.11409669069735173
0.09531316114840394
0.07959159130469828
0.0664394425849153
0.05544187213526186
0.046249866116276905
0.03857007502061549
0.03215614214250451
0.02680132915808997
0.022332260469035248
0.01860362731937215
0.015493711914409403
0.012900609877462443
0.010739045909524236
0.008937692293969235
0.0074369129159777
0.006186866831939142
0.005145915269380907
0.004279284416512041
0.003557943631015803
0.0029576649114062548
0.002458234769939338
0.002042794148032101


array([[3, 0, 2, 3, 4, 4],
       [0, 4, 2, 3, 1, 1],
       [3, 0, 2, 3, 2, 3],
       [0, 2, 2, 2, 2, 3],
       [1, 4, 4, 4, 0, 4]], dtype=int64)

In [ ]:
grid = np.arange(900).reshape((30,30))
x = np.argwhere(grid == state)[0][0]

In [295]:

#Translation Matrix
state_index = np.arange(900).reshape((30,30))
state_index[6][13]

193

In [301]:
env.P[193]

{0: [([0.25, 0.25, 0.25, 0.25, 0.25],
   [7, 14, 19, 12, 13],
   [-1, -1, -1, -1, -1],
   False)],
 1: [([0.25, 0.25, 0.25, 0.25, 0.25],
   [217, 224, 229, 222, 223],
   [-50, -1, -1, -1, -1],
   False)],
 2: [([0.25, 0.25, 0.25, 0.25, 0.25],
   [367, 374, 379, 372, 373],
   [-1, -1, -1, -50, -1],
   False)],
 3: [([0.25, 0.25, 0.25, 0.25, 0.25],
   [187, 194, 199, 192, 193],
   [-1, -1, -1, -1, -1],
   False)],
 4: [([0.25, 0.25, 0.25, 0.25, 0.25],
   [187, 194, 199, 192, 193],
   [-1, -1, -1, -1, -1],
   False)]}

In [116]:
prob, next_state, reward, done = env.P[840][4][0]

In [289]:
def value_iteration(env, theta=0.0001, discount_factor=0.8):
    """
    Value Iteration Algorithm.
    
    Args:
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.
    """
    
    def one_step_lookahead(state, V):
        """
        Helper function to calculate the value for all action in a given state.
        
        Args:
            state: The state to consider (int)
            V: The value to use as an estimator, Vector of length env.nS
        
        Returns:
            A vector of length env.nA containing the expected value of each action.
        """
        A = np.zeros(env.nA)
        for a in range(env.nA):
            for prob, next_state, reward, done in env.P[state][a]:
                #Prob a vector for the probabilities of the nex_state
                #Prob a vector for the possible next states
                # Rewards in thoose possible next states
               
                for i in range(len(prob)):
                    A[a] += prob[i] * (reward[i] + discount_factor * V[next_state[i]])
        
               
        return A
    
    V = np.zeros(env.nS)
    maxIter = 50
    i = 0
    while True:
        # Stopping condition
        delta = 0
        # Update each state...
        for s in range(env.nS):
            
            # Do a one-step lookahead to find the best action
            A = one_step_lookahead(s, V)
            best_action_value = np.max(A)
            # Calculate delta across all states seen so far
            delta = max(delta, np.abs(best_action_value - V[s]))
            # Update the value function. Ref: Sutton book eq. 4.10. 
            V[s] = best_action_value        
        # Check if we can stop 
        
        i +=1
        print(delta)
        if delta < theta:
            break
        if i >=maxIter:
            break
        
    
    # Create a deterministic policy using the optimal value function
    policy = np.zeros([env.nS, env.nA])
    for s in range(env.nS):
        # One step lookahead to find the best action for this state
        A = one_step_lookahead(s, V)
        best_action = np.argmax(A)
        # Always take the best action
        policy[s, best_action] = 1.0
    
    return policy, V

In [249]:
from random import randint

# WORK IN PROGRESS

def simulation(policy,T, start_states):
    
    
    """Simulating a policy in the Random Minotaur world. 
    Args : 
        policy - Policy in matrix form
        T - maximum time allowed in the maze
        start_states - A tuple (agent_state, minataur state) of the start state of the agent and the minotaur
    
    Returns : 
        A list of tupels [(agent_state, minataur state)] of the moves made by the agent and the minataur. 
        If_sucessful - Booelan if the agent sucessfully exit the maze before T
    """
    t = 0
    
    is_sucessful = False
    
    current_agent_state = start_states[0]
    current_minataur_state = start_states[1]
    
    while t <= T:
        minataur_next_move = randint(0,3)
        
        # minataur_state = 
        
        
        agnet_next_move = policy[agent_state, minataur_state]
        
        #If the agent in the next move reaches the terminal state
        if agent_next_move == 28:
            is_sucessful = True
        
    
    return is_sucessful
        
    
    
    
    
    
    
    
    
    
    
    

SyntaxError: invalid syntax (<ipython-input-249-582641cb258a>, line 1)